<a href="https://colab.research.google.com/github/CASAttackZW2025/CAS502Project/blob/main/CAS502ProcessingV002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpy

import simpy
import heapq
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import logging
from collections import defaultdict, deque
from itertools import combinations, product
import time
import datetime
import calendar
import csv
from typing import Dict, List, Tuple, Any, Optional

logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
logger = logging.getLogger(__name__)

class ProcessSimulation:
    """
    A manufacturing process simulation using SimPy.
    """
    def __init__(
        self,
        initial_entities,
        unit_arrival_pattern,
        paths_info,
        p_level_changers=None,
        path_changers=None,
        priority_changers=None,
        exclude_processors=None,
        n_entities=100,
        default_p_level=1,
        default_entity_path='Path1',
        default_priority=10,
        top_n=10
    ):
        self.env = simpy.Environment()
        self.initial_entities = initial_entities
        self.unit_arrival_pattern = unit_arrival_pattern
        self.paths_info = paths_info
        self.p_level_changers = p_level_changers or {}
        self.path_changers = path_changers or {}
        self.priority_changers = priority_changers or {}
        self.exclude_processors = exclude_processors or []
        self.n_entities = n_entities
        self.default_p_level = default_p_level
        self.default_entity_path = default_entity_path
        self.default_priority = default_priority
        self.top_n = top_n
        self.G_all = self.compose_graphs()
        self.processor_capacities, self.processor_to_subprocessors, self.subprocessor_to_processor = self.set_processor_capacities()
        self.processor_resources = {}
        for proc, capacity in self.processor_capacities.items():
            self.processor_resources[proc] = simpy.PriorityResource(self.env, capacity=capacity)
        self.entity_attributes = {}
        self.entity_processor_history = defaultdict(lambda: defaultdict(list))
        self.start_times = defaultdict(dict)
        self.finish_times = defaultdict(dict)
        self.busy_periods = defaultdict(list)
        self.entity_id_counter = 1
        self.total_entities_generated = 0
        self.results = None

    def compose_graphs(self):
        """
        Combine all path graphs into a single directed graph.
        """
        G_all = nx.DiGraph()
        for path_name, path_data in self.paths_info.items():
            path_graph = path_data['graph']
            G_all = nx.compose(G_all, path_graph)
        return G_all

    def set_processor_capacities(self):
        """
        Set capacities for processors and create subprocessors for capacities > 1.
        """
        processor_capacities = {}
        processor_to_subprocessors = {}
        subprocessor_to_processor = {}

        for path_name, path_data in self.paths_info.items():
            processing_times = path_data.get('processing_times', {})
            for proc, proc_info in processing_times.items():
                capacity = proc_info.get('capacity', 1)
                processor_capacities[proc] = capacity
                if capacity > 1:
                    subprocessor_names = [f"{proc}_{i+1}" for i in range(capacity)]
                    processor_to_subprocessors[proc] = subprocessor_names
                    for subproc in subprocessor_names:
                        subprocessor_to_processor[subproc] = proc
                        processor_capacities[subproc] = 1
                else:
                    processor_to_subprocessors[proc] = [proc]
                    subprocessor_to_processor[proc] = proc
                    processor_capacities[proc] = 1

        return processor_capacities, processor_to_subprocessors, subprocessor_to_processor

    def get_processing_time(self, proc_info, current_p_level):
        """
        Determine processing time based on current p-level.
        """
        if 'processing_time' not in proc_info:
            raise KeyError("proc_info must contain a 'processing_time' key.")
        processing_time = proc_info['processing_time']
        if isinstance(processing_time, dict):
            return processing_time.get(current_p_level, processing_time.get(1, 0))
        return processing_time

    def generate_entities(self):
        """
        Generate all entities based on initial entities and arrival pattern.
        """
        for initial_info in self.initial_entities:
            arrival_time = initial_info.get('arrival_time', 0)
            num_entities = initial_info['num_entities']
            if self.total_entities_generated + num_entities > self.n_entities:
                num_entities = self.n_entities - self.total_entities_generated
            entity_paths = initial_info.get('entity_paths', [self.default_entity_path] * num_entities)
            p_levels = initial_info.get('p_levels', [self.default_p_level] * num_entities)
            priorities = initial_info.get('priorities', [self.default_priority] * num_entities)
            for i in range(num_entities):
                entity_id = self.entity_id_counter
                self.entity_id_counter += 1
                self.total_entities_generated += 1
                self.entity_attributes[entity_id] = {
                    'arrival_time': arrival_time,
                    'path': entity_paths[i % len(entity_paths)],
                    'p_level': p_levels[i % len(p_levels)],
                    'priority': priorities[i % len(priorities)]
                }
                self.env.process(self.entity_arrival_process(entity_id, arrival_time))
            if self.total_entities_generated >= self.n_entities:
                break
        if self.total_entities_generated < self.n_entities and self.unit_arrival_pattern:
            repetition = 0
            max_arrival_time = max(info['arrival_time'] for info in self.unit_arrival_pattern) if self.unit_arrival_pattern else 0
            while self.total_entities_generated < self.n_entities:
                for arrival_info in self.unit_arrival_pattern:
                    arrival_time = arrival_info['arrival_time'] + repetition * (max_arrival_time + 1)
                    num_entities = arrival_info['num_entities']
                    if self.total_entities_generated + num_entities > self.n_entities:
                        num_entities = self.n_entities - self.total_entities_generated
                    entity_paths = arrival_info.get('entity_paths', [self.default_entity_path] * num_entities)
                    p_levels = arrival_info.get('p_levels', [self.default_p_level] * num_entities)
                    priorities = arrival_info.get('priorities', [self.default_priority] * num_entities)
                    for i in range(num_entities):
                        entity_id = self.entity_id_counter
                        self.entity_id_counter += 1
                        self.total_entities_generated += 1
                        self.entity_attributes[entity_id] = {
                            'arrival_time': arrival_time,
                            'path': entity_paths[i % len(entity_paths)],
                            'p_level': p_levels[i % len(p_levels)],
                            'priority': priorities[i % len(priorities)]
                        }
                        self.env.process(self.entity_arrival_process(entity_id, arrival_time))
                    if self.total_entities_generated >= self.n_entities:
                        break
                repetition += 1
                if self.total_entities_generated >= self.n_entities:
                    break

    def entity_arrival_process(self, entity_id, arrival_time):
        """
        Process representing an entity's arrival and path through the system.
        """
        yield self.env.timeout(arrival_time)
        entity = self.entity_attributes[entity_id]
        entity_path = entity['path']
        first_processors = [p for p in self.G_all.nodes if self.G_all.in_degree(p) == 0]
        for proc in first_processors:
            if proc in self.paths_info[entity_path]['processing_times']:
                self.env.process(self.process_at_processor(entity_id, proc))

    def process_at_processor(self, entity_id, proc):
        """
        Process an entity at a specified processor and handle transitions to successors.
        """
        entity = self.entity_attributes[entity_id]
        entity_path = entity['path']
        current_p_level = entity['p_level']
        priority = entity['priority']
        if proc not in self.paths_info[entity_path]['processing_times']:
            return
        proc_info = self.paths_info[entity_path]['processing_times'][proc]
        subprocessors = self.processor_to_subprocessors.get(proc, [proc])
        while True:
            for subproc in subprocessors:
                if not self.processor_resources[subproc].queue and self.processor_resources[subproc].count < self.processor_resources[subproc].capacity:
                    resource = self.processor_resources[subproc]
                    with resource.request(priority=priority) as req:
                        yield req
                        start_time = self.env.now
                        self.start_times[subproc][entity_id] = start_time
                        original_proc = self.subprocessor_to_processor.get(subproc, subproc)
                        if original_proc not in self.entity_processor_history[entity_id]:
                            self.entity_processor_history[entity_id][original_proc] = []
                        self.entity_processor_history[entity_id][original_proc].append({'start_time': start_time})
                        processing_time = self.get_processing_time(proc_info, current_p_level)
                        yield self.env.timeout(processing_time)
                        finish_time = self.env.now
                        self.finish_times[subproc][entity_id] = finish_time
                        self.entity_processor_history[entity_id][original_proc][-1]['end_time'] = finish_time
                        self.busy_periods[subproc].append((start_time, finish_time))
                        if original_proc in self.p_level_changers:
                            entity['p_level'] = self.p_level_changers[original_proc](entity['p_level'])
                        if original_proc in self.path_changers:
                            entity['path'] = self.path_changers[original_proc](entity['path'])
                        if original_proc in self.priority_changers:
                            entity['priority'] = self.priority_changers[original_proc](entity['priority'])
                        self.schedule_successor_processors(entity_id, original_proc)
                    return
            yield self.env.timeout(0.1)

    def schedule_successor_processors(self, entity_id, proc):
        """
        Determine and schedule next processors for the entity.
        """
        entity = self.entity_attributes[entity_id]
        entity_path = entity['path']
        current_p_level = entity['p_level']
        proc_info = self.paths_info[entity_path]['processing_times'].get(proc, {})
        if 'p_level_successors' in proc_info:
            possible_successors = proc_info['p_level_successors'].get(current_p_level, [])
            for succ in possible_successors:
                self.env.process(self.process_at_processor(entity_id, succ))
        else:
            for succ in self.G_all.successors(proc):
                if succ in self.paths_info[entity_path]['processing_times']:
                    preds = list(self.G_all.predecessors(succ))
                    all_preds_finished = True
                    for pred in preds:
                        subprocs = self.processor_to_subprocessors.get(pred, [pred])
                        pred_finished = False
                        for subproc in subprocs:
                            if entity_id in self.finish_times.get(subproc, {}):
                                pred_finished = True
                                break
                        if not pred_finished:
                            all_preds_finished = False
                            break
                    if all_preds_finished:
                        self.env.process(self.process_at_processor(entity_id, succ))

    def run(self):
        """
        Run the simulation and collect results.
        """
        self.generate_entities()
        self.env.run()
        total_time = 0
        for proc in self.finish_times:
            if self.finish_times[proc]:
                total_time = max(total_time, max(self.finish_times[proc].values()))
        idle_times, busy_times = self.calculate_idle_busy_times(total_time)
        all_processors = {proc: {'busy_periods': self.busy_periods[proc]} for proc in self.busy_periods}
        joint_times = self.compute_joint_times(all_processors, k=2, total_time=total_time)
        top_idle_combinations = self.find_top_idle_combinations(joint_times, k=2, top_n=self.top_n)
        self.results = {
            'start_times': self.start_times,
            'finish_times': self.finish_times,
            'idle_times': idle_times,
            'busy_times': busy_times,
            'entity_attributes': self.entity_attributes,
            'total_processing_time': total_time,
            'busy_periods': self.busy_periods,
            'joint_times': joint_times,
            'paths_info': self.paths_info,
            'G_all': self.G_all,
            'entity_processor_history': self.entity_processor_history,
            'top_idle_combinations': top_idle_combinations
        }
        return self.results

    def calculate_idle_busy_times(self, total_time):
        """
        Calculate idle and busy times for each processor.
        """
        idle_times = defaultdict(float)
        busy_times = defaultdict(float)
        for proc in self.finish_times:
            sorted_busy_periods = sorted(self.busy_periods[proc])
            idle_periods = self.get_idle_periods(sorted_busy_periods, total_time)
            idle_time = sum(end - start for start, end in idle_periods)
            busy_time = sum(end - start for start, end in sorted_busy_periods)
            idle_times[proc] = idle_time
            busy_times[proc] = busy_time
        return idle_times, busy_times

    def get_idle_periods(self, busy_periods, total_time):
        """
        Calculate idle periods based on busy periods.
        """
        for period in busy_periods:
            if not isinstance(period, tuple) or len(period) != 2:
                raise ValueError(f"Invalid busy period format: {period}. Expected a tuple of (start, end).")
            start, finish = period
            if start > finish:
                raise ValueError(f"Invalid busy period times: {period}. Start time must be equal or less than end time.")
        idle_periods = []
        prev_finish = 0
        for start, finish in sorted(busy_periods):
            if start > prev_finish:
                idle_periods.append((prev_finish, start))
            prev_finish = max(prev_finish, finish)
        if prev_finish < total_time:
            idle_periods.append((prev_finish, total_time))
        return idle_periods

    def intersect_intervals(self, intervals1, intervals2):
        """
        Find overlapping intervals between two lists of time periods.
        """
        for interval in intervals1 + intervals2:
            if len(interval) != 2 or interval[0] >= interval[1]:
                raise ValueError(f"Invalid interval: {interval}. Each interval must be a tuple of (start, end) with start < end.")
        result = []
        i, j = 0, 0
        while i < len(intervals1) and j < len(intervals2):
            a_start, a_end = intervals1[i]
            b_start, b_end = intervals2[j]
            start = max(a_start, b_start)
            end = min(a_end, b_end)
            if start < end:
                result.append((start, end))
            if a_end < b_end:
                i += 1
            else:
                j += 1
        return result

    def compute_joint_times(self, all_processors, k, total_time):
        """
        Compute joint idle/busy times for combinations of processors.
        """
        joint_times = []
        processor_list = [proc for proc in all_processors.keys() if proc not in self.exclude_processors]
        for size in range(1, k + 1):
            for combo in combinations(processor_list, size):
                states = ['busy', 'idle']
                for state_combo in product(states, repeat=size):
                    proc_state = dict(zip(combo, state_combo))
                    intervals = [(0, total_time)]
                    for proc in combo:
                        if proc_state[proc] == 'busy':
                            periods = all_processors[proc]['busy_periods']
                        else:
                            periods = self.get_idle_periods(all_processors[proc]['busy_periods'], total_time)
                        intervals = self.intersect_intervals(intervals, periods)
                        if not intervals:
                            break
                    total_interval = sum(end - start for start, end in intervals)
                    if total_interval > 0:
                        joint_times.append({
                            'processors': proc_state,
                            'total_time': total_interval,
                            'intervals': intervals
                        })
        return joint_times

    def collect_all_processors(self, joint_times):
        """
        Collect all unique processors from joint_times.
        """
        all_procs = set()
        for jt in joint_times:
            all_procs.update(jt['processors'].keys())
        return all_procs

    def filter_processors(self, processor_list, exclude_processors):
        """
        Filter out excluded processors from the processor list.
        """
        exclude_set = set(exclude_processors)
        logger.info(f"Excluding processors: {exclude_set}")
        filtered_processors = [proc for proc in processor_list if proc not in exclude_set]
        logger.info(f"Filtered processors: {filtered_processors}")
        return filtered_processors

    def compute_total_time(self, joint_times, combo, proc_state):
        """
        Compute the total time where processors are in the specified states.
        """
        total_time = 0.0
        for jt in joint_times:
            jt_procs = jt.get('processors', {})
            jt_total_time = jt.get('total_time', 0.0)
            if set(jt_procs.keys()) == set(combo):
                if all(jt_procs[proc] == state for proc, state in proc_state.items()):
                    total_time += jt_total_time
        return total_time

    def find_top_idle_combinations(self, joint_times, k, top_n=10):
        """
        Find the top N combinations of processors with maximum joint idle time.
        """
        combination_times = {}
        all_procs = self.collect_all_processors(joint_times)
        processor_list = self.filter_processors(all_procs, self.exclude_processors)
        for size in range(2, k + 1):
            for combo in combinations(processor_list, size):
                for busy_proc in combo:
                    proc_state = {proc: 'busy' if proc == busy_proc else 'idle' for proc in combo}
                    total_time = self.compute_total_time(joint_times, combo, proc_state)
                    if total_time > 0:
                        key = (tuple(sorted(combo)), busy_proc)
                        combination_times[key] = combination_times.get(key, 0) + total_time
        sorted_combinations = sorted(combination_times.items(), key=lambda item: item[1], reverse=True)
        top_combinations = sorted_combinations[:top_n]
        result = []
        for (procs, busy_proc), total_time in top_combinations:
            result.append({
                'processors': procs,
                'busy_processor': busy_proc,
                'total_time': total_time
            })
        return result

    def plot_processing_schedule(self):
        """
        Plot the processing schedule for each processor.
        """
        if not self.results:
            logger.warning("No results available. Run the simulation first.")
            return
        entity_processor_history = self.results['entity_processor_history']
        processors = set()
        entities = set()
        for entity_id in entity_processor_history:
            entities.add(entity_id)
            processors.update(entity_processor_history[entity_id].keys())
        entities = sorted(entities)
        n_entities = len(entities)
        processor_start_times = {}
        for processor in processors:
            earliest_start = float('inf')
            for entity_id in entity_processor_history:
                if processor in entity_processor_history[entity_id]:
                    for visit in entity_processor_history[entity_id][processor]:
                        s = visit['start_time']
                        if s < earliest_start:
                            earliest_start = s
            processor_start_times[processor] = earliest_start
        processors = [p for p in processors if p != "Start_Job"]
        sorted_processors = sorted(processors, key=lambda p: processor_start_times[p])
        entity_colors = self.generate_random_colors(n_entities)
        entity_color_dict = {entity_id: entity_colors[idx] for idx, entity_id in enumerate(entities)}
        processor_colors = plt.cm.get_cmap('tab20', len(sorted_processors))
        processor_color_dict = {processor: processor_colors(idx % 20) for idx, processor in enumerate(sorted_processors)}
        fig, ax = plt.subplots(figsize=(14, 8))
        yticks = []
        ytick_labels = []
        y_base = 0
        for processor in sorted_processors:
            visits = []
            for entity_id in entity_processor_history:
                if processor in entity_processor_history[entity_id]:
                    for visit in entity_processor_history[entity_id][processor]:
                        s = visit['start_time']
                        f = visit.get('end_time', visit['start_time'])
                        visits.append({'entity_id': entity_id, 'start_time': s, 'end_time': f})
            visits.sort(key=lambda v: v['start_time'])
            capacity = self.processor_capacities.get(processor, 1)
            lanes = [[] for _ in range(capacity)]
            for visit in visits:
                placed = False
                for lane in lanes:
                    if not lane or visit['start_time'] >= lane[-1]['end_time']:
                        lane.append(visit)
                        placed = True
                        break
                if not placed:
                    lanes.append([visit])
            num_lanes = len(lanes)
            for lane_idx, lane in enumerate(lanes):
                for visit in lane:
                    s = visit['start_time']
                    f = visit['end_time']
                    entity_id = visit['entity_id']
                    entity_color = entity_color_dict[entity_id]
                    processor_color = processor_color_dict[processor]
                    if n_entities < 22:
                        ax.broken_barh(
                            [(s, f - s)],
                            (y_base + lane_idx - 0.4, 1.3),
                            facecolors=processor_color,
                            edgecolors='black',
                            linewidth=1
                        )
                        ax.broken_barh(
                            [(s, f - s)],
                            (y_base + lane_idx, 0.4),
                            facecolors=entity_color,
                            alpha=1,
                            edgecolors='black',
                            linewidth=1
                        )
                    else:
                        ax.broken_barh(
                            [(s, f - s)],
                            (y_base + lane_idx - 0.4, 1.3),
                            facecolors=processor_color,
                            edgecolors='none',
                            linewidth=1
                        )
                        ax.broken_barh(
                            [(s, f - s)],
                            (y_base + lane_idx, 0.4),
                            facecolors=entity_color,
                            alpha=1,
                            edgecolors='none'
                        )
                    if n_entities < 11:
                        ax.text(
                            s + (f - s) / 2,
                            y_base + lane_idx + 0.1,
                            f'{entity_id}',
                            ha='center',
                            va='center',
                            fontsize=8,
                            color='white'
                        )
            ytick_pos = y_base + (num_lanes - 1) / 2
            yticks.append(ytick_pos)
            ytick_labels.append(f"{processor}")
            y_base += num_lanes + 1
        ax.set_xlabel('Time (hours)')
        ax.set_ylabel('Processors')
        ax.set_yticks(yticks)
        ax.set_yticklabels(ytick_labels)
        ax.set_title('Processing Schedule of Entities at Each Processor')
        ax.grid(True, axis='x', linestyle='--', alpha=0.7)
        from matplotlib.patches import Patch
        processor_patches = [
            Patch(facecolor=processor_color_dict[processor], edgecolor='black', label=f'{processor}')
            for processor in sorted_processors
        ]
        ax.legend(handles=processor_patches, title='Processors', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.show()

    def generate_random_colors(self, num_colors):
        """
        Generate random colors for visualization.
        """
        np.random.seed(42)
        colors = []
        for _ in range(num_colors):
            hue = np.random.rand()
            saturation = 0.5 + 0.5 * np.random.rand()
            value = 0.5 + 0.5 * np.random.rand()
            from matplotlib import colors as mcolors
            rgb_color = mcolors.hsv_to_rgb((hue, saturation, value))
            colors.append(rgb_color)
        return colors

    def plot_results(self):
        """
        Plot the arrival times and finish times of entities at the last processor of each path.
        """
        if not self.results:
            logger.warning("No results available. Run the simulation first.")
            return
        entity_attributes = self.results['entity_attributes']
        finish_times = self.results['finish_times']
        paths_info = self.results['paths_info']
        plt.figure(figsize=(12, 6))
        markers = ['o', 's', '^', 'D', 'v', 'P', '*', 'X', '+', '<', '>']
        entities = list(entity_attributes.keys())
        entities.sort()
        num_entities = len(entities)
        marker_idx = 0
        color_cycle = plt.cm.get_cmap('tab20')
        for idx, (path_name, path_data) in enumerate(paths_info.items()):
            path_entities = [eid for eid, attr in entity_attributes.items() if attr['path'] == path_name]
            if not path_entities:
                continue
            arrival_times = [entity_attributes[eid]['arrival_time'] for eid in path_entities]
            if num_entities < 49:
                plt.plot(
                    path_entities,
                    arrival_times,
                    label=f'Arrival Times {path_name}',
                    marker=markers[marker_idx % len(markers)],
                    linestyle='--',
                    color=color_cycle(idx % 20)
                )
            else:
                plt.plot(
                    path_entities,
                    arrival_times,
                    label=f'Arrival Times {path_name}',
                    linestyle='--',
                    color=color_cycle(idx % 20)
                )
            marker_idx += 1
            G = path_data['graph']
            try:
                processors = list(nx.topological_sort(G))
            except nx.NetworkXUnfeasible:
                processors = list(G.nodes())
            last_processor = None
            for proc in reversed(processors):
                if proc in path_data['processing_times']:
                    last_processor = proc
                    break
            if last_processor:
                subprocessors = self.processor_to_subprocessors.get(last_processor, [last_processor])
                finish_times_last_proc = []
                for eid in path_entities:
                    finish_time = None
                    for subproc in subprocessors:
                        if eid in finish_times.get(subproc, {}):
                            finish_time = finish_times[subproc][eid]
                            break
                    finish_times_last_proc.append(finish_time if finish_time is not None else np.nan)
                if num_entities < 49:
                    plt.plot(
                        path_entities,
                        finish_times_last_proc,
                        label=f'Finish Times at {last_processor}',
                        marker=markers[marker_idx % len(markers)],
                        linestyle='-',
                        color=color_cycle((idx + 1) % 20)
                    )
                else:
                    plt.plot(
                        path_entities,
                        finish_times_last_proc,
                        label=f'Finish Times at {last_processor}',
                        linestyle='-',
                        color=color_cycle((idx + 1) % 20)
                    )
                marker_idx += 1
        plt.xlabel('Entity ID')
        plt.ylabel('Time (hours)')
        plt.title('Processing Times of Entities at Last Processor of Each Path')
        plt.legend()
        plt.grid(True)
        plt.show()

    def plot_results_individual_processors(self):
        """
        Plot the arrival times and finish times of entities at each processor.
        """
        if not self.results:
            logger.warning("No results available. Run the simulation first.")
            return
        entity_attributes = self.results['entity_attributes']
        finish_times = self.results['finish_times']
        paths_info = self.results['paths_info']
        plt.figure(figsize=(14, 8))
        markers = ['o', 's', '^', 'D', 'v', 'P', '*', 'X', '+', '<', '>']
        color_cycle = plt.cm.get_cmap('tab20')
        entities = list(entity_attributes.keys())
        entities.sort()
        num_entities = len(entities)
        marker_idx = 0
        for idx, (path_name, path_data) in enumerate(paths_info.items()):
            path_entities = [eid for eid, attr in entity_attributes.items() if attr['path'] == path_name]
            if not path_entities:
                continue
            arrival_times = [entity_attributes[eid]['arrival_time'] for eid in path_entities]
            if num_entities < 49:
                plt.plot(
                    path_entities,
                    arrival_times,
                    label=f'Arrival Times {path_name}',
                    marker=markers[marker_idx % len(markers)],
                    linestyle='--',
                    color=color_cycle(idx % 20)
                )
            else:
                plt.plot(
                    path_entities,
                    arrival_times,
                    label=f'Arrival Times {path_name}',
                    linestyle='--',
                    color=color_cycle(idx % 20)
                )
            marker_idx += 1
            G = path_data['graph']
            try:
                processors = list(nx.topological_sort(G))
            except nx.NetworkXUnfeasible:
                processors = list(G.nodes())
            for proc_idx, processor in enumerate(processors):
                if processor in path_data['processing_times']:
                    subprocessors = self.processor_to_subprocessors.get(processor, [processor])
                    finish_times_proc = []
                    for eid in path_entities:
                        finish_time = None
                        for subproc in subprocessors:
                            if eid in finish_times.get(subproc, {}):
                                finish_time = finish_times[subproc][eid]
                                break
                        finish_times_proc.append(finish_time if finish_time is not None else np.nan)
                    if num_entities < 49:
                        plt.plot(
                            path_entities,
                            finish_times_proc,
                            label=f'Finish Times at {processor}',
                            marker=markers[marker_idx % len(markers)],
                            linestyle='-',
                            color=color_cycle((idx + proc_idx + 1) % 20)
                        )
                    else:
                        plt.plot(
                            path_entities,
                            finish_times_proc,
                            label=f'Finish Times at {processor}',
                            linestyle='-',
                            color=color_cycle((idx + proc_idx + 1) % 20)
                        )
                    marker_idx += 1
        plt.xlabel('Entity ID')
        plt.ylabel('Time (hours)')
        plt.title('Processing Times of Entities at Each Processor')
        plt.legend()
        plt.grid(True)
        plt.show()

    def save_joint_times_to_csv(self, output_file='idle_combinations.csv'):
        """
        Save joint idle/busy times to a CSV file with dynamically determined headers.
        """
        if not self.results:
            logger.warning("No results available. Run the simulation first.")
            return
        joint_times = self.results['joint_times']
        if not joint_times:
            print("No joint times to save.")
            return
        try:
            max_busy = max(len([proc for proc, state in jt['processors'].items() if state == 'busy']) for jt in joint_times)
            max_idle = max(len([proc for proc, state in jt['processors'].items() if state == 'idle']) for jt in joint_times)
            max_total = max(len(jt['processors']) for jt in joint_times)
            fieldnames = [
                'Processors',
                'Busy Processors',
                'Idle Processors',
                'Total Time'
            ]
            fieldnames += [f'Busy Processor{i+1}' for i in range(max_busy)]
            fieldnames += [f'Idle Processor{i+1}' for i in range(max_idle)]
            fieldnames += [f'Processor{i+1}' for i in range(max_total)]
            with open(output_file, 'w', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                for jt in joint_times:
                    processors = jt['processors']
                    busy_processors = sorted([proc for proc, state in processors.items() if state == 'busy'])
                    idle_processors = sorted([proc for proc, state in processors.items() if state == 'idle'])
                    total_time = jt['total_time']
                    if busy_processors and idle_processors:
                        row = {
                            'Processors': ', '.join(sorted(processors.keys())),
                            'Busy Processors': ', '.join(busy_processors),
                            'Idle Processors': ', '.join(idle_processors),
                            'Total Time': f"{total_time:.2f}"
                        }
                        for i in range(max_busy):
                            key = f'Busy Processor{i+1}'
                            row[key] = busy_processors[i] if i < len(busy_processors) else ''
                        for i in range(max_idle):
                            key = f'Idle Processor{i+1}'
                            row[key] = idle_processors[i] if i < len(idle_processors) else ''
                        sorted_procs = sorted(processors.keys())
                        for i in range(max_total):
                            key = f'Processor{i+1}'
                            row[key] = sorted_procs[i] if i < len(sorted_procs) else ''
                        writer.writerow(row)
            print(f"Joint times CSV file '{output_file}' has been created successfully.")
        except Exception as e:
            print(f"An error occurred while saving the joint times CSV file: {e}")

    def clean_idle_combinations_csv(self, input_file, output_file, excluded_processors):
        """
        Remove rows from the idle_combinations.csv that contain any excluded processors.
        """
        excluded_set = set(excluded_processors)
        try:
            with open(input_file, 'r') as infile, open(output_file, 'w', newline='') as outfile:
                reader = csv.DictReader(infile)
                fieldnames = reader.fieldnames
                writer = csv.DictWriter(outfile, fieldnames=fieldnames)
                writer.writeheader()
                for row in reader:
                    processors_in_row = set(proc.strip() for proc in row['Processors'].split(','))
                    if excluded_set.intersection(processors_in_row):
                        continue
                    writer.writerow(row)
            print(f"Cleaned CSV saved to '{output_file}'.")
        except Exception as e:
            print(f"An error occurred while cleaning the CSV file: {e}")

    def save_processor_schedule_and_dependencies_to_csv(self, output_file='processor_schedules_dependencies.csv'):
        """
        Save processor schedules and dependencies to a CSV file.
        """
        if not self.results:
            logger.warning("No results available. Run the simulation first.")
            return
        try:
            with open(output_file, 'w', newline='') as csvfile:
                fieldnames = ['Processor', 'Predecessors', 'Successors', 'Entity_ID', 'Start_Time', 'End_Time']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                G_all = self.results['G_all']
                entity_history = self.results['entity_processor_history']
                for proc in G_all.nodes:
                    predecessors = list(G_all.predecessors(proc))
                    successors = list(G_all.successors(proc))
                    for entity_id, proc_history in entity_history.items():
                        if proc in proc_history:
                            visits = proc_history[proc]
                            for visit in visits:
                                start_time = visit.get('start_time', '')
                                end_time = visit.get('end_time', '')
                                writer.writerow({
                                    'Processor': proc,
                                    'Predecessors': ';'.join(predecessors),
                                    'Successors': ';'.join(successors),
                                    'Entity_ID': entity_id,
                                    'Start_Time': start_time,
                                    'End_Time': end_time
                                })
            print(f"Processor schedules and dependencies have been saved to '{output_file}'.")
        except Exception as e:
            print(f"An error occurred while saving the processor schedules and dependencies CSV file: {e}")

    def construct_optimal_pools(self, file_path, k, m, excluded_processors=None):
        """
        Construct optimal processor pools based on idle times from a CSV file.
        """
        excluded_processors = excluded_processors or []
        excluded_set = set(excluded_processors)
        processor_idle_times = defaultdict(float)
        pools = defaultdict(list)
        try:
            with open(file_path, 'r') as file:
                reader = csv.DictReader(file)
                for row in reader:
                    processors_in_row = tuple(sorted(proc.strip() for proc in row['Processors'].split(',')))
                    if excluded_set.intersection(processors_in_row):
                        continue
                    try:
                        busy_processors = [proc.strip() for proc in row['Busy Processors'].split(',') if proc.strip()]
                    except KeyError:
                        busy_processors = []
                    if excluded_set.intersection(busy_processors):
                        continue
                    try:
                        total_time = float(row['Total Time'])
                    except ValueError:
                        total_time = 0.0
                    processor_idle_times[processors_in_row] += total_time
                    for busy_processor in busy_processors:
                        pools[processors_in_row].append((busy_processor, total_time))
        except Exception as e:
            print(f"An error occurred while reading the CSV file: {e}")
            return []
        sum_pools = defaultdict(float)
        for processors, times in pools.items():
            for busy_processor, total_time in times:
                subset = tuple(sorted(set(processors) - {busy_processor}))
                sum_pools[processors] += total_time
                if subset:
                    sum_pools[subset] += total_time
        def compute_idle_times(processors, depth, current_time):
            total_idle_time = current_time
            for i in range(1, len(processors)):
                for subset in combinations(processors, i):
                    if subset in sum_pools:
                        total_idle_time += compute_idle_times(subset, depth + 1, sum_pools[subset])
            return total_idle_time
        optimal_pools = []
        for processors in sum_pools.keys():
            if len(processors) <= k:
                total_idle_time = compute_idle_times(processors, 0, sum_pools[processors])
                optimal_pools.append((processors, total_idle_time))
        optimal_pools.sort(key=lambda x: x[1], reverse=True)
        return optimal_pools[:m]

    def save_optimal_pools_to_csv(self, optimal_pools, output_file='optimal_pools.csv'):
        """
        Save optimal pools to a CSV file.
        """
        if not optimal_pools:
            print("No optimal pools to save.")
            return
        try:
            max_processors = max(len(pool[0]) for pool in optimal_pools)
            fieldnames = ['Pool', 'Total Idle Time']
            fieldnames += [f'Processor{i+1}' for i in range(max_processors)]
            with open(output_file, 'w', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                for i, (processors, total_idle_time) in enumerate(optimal_pools, start=1):
                    row = {
                        'Pool': i,
                        'Total Idle Time': f"{total_idle_time:.2f} hours"
                    }
                    for j, processor in enumerate(processors):
                        row[f'Processor{j+1}'] = processor
                    for j in range(len(processors), max_processors):
                        row[f'Processor{j+1}'] = ''
                    writer.writerow(row)
            print(f"CSV file '{output_file}' has been created successfully.")
        except Exception as e:
            print(f"An error occurred while saving the CSV file: {e}")

    def create_entity_arrival_schedule(
        self,
        start_date: str,
        interval_type: str,
        end_date: str,
        initial_num_entities: int,
        arrival_pattern_num_entities: int,
        remove_weekends: bool = False,
        holiday_dates: Optional[List[str]] = None,
        subtract_pto_hours_per_week: float = 0,
        scale_days: float = 24,
        remove_holiday_overlap: bool = True
    ) -> Dict[str, Any]:
        """
        Create a schedule of entity arrivals based on specified parameters.
        """
        start_date_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d').date()
        end_date_dt = datetime.datetime.strptime(end_date, '%Y-%m-%d').date()
        if holiday_dates:
            holiday_dates_dt = [datetime.datetime.strptime(date, '%Y-%m-%d').date() for date in holiday_dates]
        else:
            holiday_dates_dt = []
        initial_entities = [{
            'arrival_time': 0,
            'num_entities': initial_num_entities
        }]
        unit_arrival_pattern = []
        def add_interval(date: datetime.date, interval: str) -> datetime.date:
            if interval == 'daily':
                return date + datetime.timedelta(days=1)
            elif interval == 'weekly':
                return date + datetime.timedelta(weeks=1)
            elif interval == 'monthly':
                month = date.month
                year = date.year + month // 12
                month = month % 12 + 1
                day = min(date.day, calendar.monthrange(year, month)[1])
                return datetime.date(year, month, day)
            elif interval == 'quarterly':
                month = date.month + 3
                year = date.year + (month - 1) // 12
                month = (month - 1) % 12 + 1
                day = min(date.day, calendar.monthrange(year, month)[1])
                return datetime.date(year, month, day)
            elif interval == 'yearly':
                try:
                    return datetime.date(date.year + 1, date.month, date.day)
                except ValueError:
                    return datetime.date(date.year + 1, date.month, 28)
            else:
                raise ValueError("Invalid interval type. Choose from 'daily', 'weekly', 'monthly', 'quarterly', 'yearly'.")
        def is_weekend(date: datetime.date) -> bool:
            return date.weekday() >= 5
        def calculate_total_hours(date: datetime.date, interval: str) -> float:
            if interval == 'daily':
                return scale_days
            elif interval == 'weekly':
                return scale_days * 7
            elif interval == 'monthly':
                _, num_days = calendar.monthrange(date.year, date.month)
                return scale_days * num_days
            elif interval == 'quarterly':
                total_hours = 0
                for m in range(0, 3):
                    month = date.month + m
                    year = date.year + (month - 1) // 12
                    month = (month - 1) % 12 + 1
                    _, num_days = calendar.monthrange(year, month)
                    total_hours += scale_days * num_days
                return total_hours
            elif interval == 'yearly':
                return scale_days * 366 if calendar.isleap(date.year) else scale_days * 365
            else:
                return scale_days
        def calculate_weekend_hours(date: datetime.date, interval: str) -> float:
            weekend_hours = 0
            if interval == 'daily':
                if is_weekend(date):
                    weekend_hours += scale_days
            elif interval == 'weekly':
                weekend_hours += scale_days * 2
            elif interval in ['monthly', 'quarterly', 'yearly']:
                temp_date = date
                next_date = add_interval(temp_date, interval)
                while temp_date < next_date:
                    if is_weekend(temp_date):
                        weekend_hours += scale_days
                    temp_date += datetime.timedelta(days=1)
            return weekend_hours
        def calculate_holiday_hours(date: datetime.date, interval: str) -> float:
            holiday_hours = 0
            if not holiday_dates_dt:
                return holiday_hours
            if interval == 'daily':
                if date in holiday_dates_dt:
                    holiday_hours += scale_days
            else:
                next_date = add_interval(date, interval)
                for holiday in holiday_dates_dt:
                    if date <= holiday < next_date:
                        if remove_holiday_overlap or not is_weekend(holiday):
                            holiday_hours += scale_days
            return holiday_hours
        def calculate_pto_hours(interval: str) -> float:
            if subtract_pto_hours_per_week <= 0:
                return 0
            if interval == 'daily':
                return (subtract_pto_hours_per_week / 7) * (scale_days / 24)
            elif interval == 'weekly':
                return subtract_pto_hours_per_week * (scale_days / 24)
            elif interval == 'monthly':
                weeks_in_month = 4.345
                return subtract_pto_hours_per_week * weeks_in_month * (scale_days / 24)
            elif interval == 'quarterly':
                weeks_in_quarter = 13
                return subtract_pto_hours_per_week * weeks_in_quarter * (scale_days / 24)
            elif interval == 'yearly':
                weeks_in_year = 52
                return subtract_pto_hours_per_week * weeks_in_year * (scale_days / 24)
            else:
                return 0
        current_date = start_date_dt
        cumulative_hours = 0
        total_adjusted_hours = 0
        total_hours = calculate_total_hours(current_date, interval_type)
        adjusted_hours = total_hours
        if remove_weekends:
            weekend_hours = calculate_weekend_hours(current_date, interval_type)
            adjusted_hours -= weekend_hours
        holiday_hours = calculate_holiday_hours(current_date, interval_type)
        adjusted_hours -= holiday_hours
        pto_hours = calculate_pto_hours(interval_type)
        adjusted_hours -= pto_hours
        adjusted_hours = max(adjusted_hours, 0)
        cumulative_hours += adjusted_hours
        current_date = add_interval(current_date, interval_type)
        while current_date <= end_date_dt:
            total_hours = calculate_total_hours(current_date, interval_type)
            adjusted_hours = total_hours
            if remove_weekends:
                weekend_hours = calculate_weekend_hours(current_date, interval_type)
                adjusted_hours -= weekend_hours
            holiday_hours = calculate_holiday_hours(current_date, interval_type)
            adjusted_hours -= holiday_hours
            pto_hours = calculate_pto_hours(interval_type)
            adjusted_hours -= pto_hours
            adjusted_hours = max(adjusted_hours, 0)
            unit_arrival_pattern.append({
                'arrival_time': cumulative_hours,
                'num_entities': arrival_pattern_num_entities
            })
            cumulative_hours += adjusted_hours
            total_adjusted_hours += adjusted_hours
            if end_date_dt <= add_interval(current_date, interval_type):
                new_date2 = current_date
                new_date1 = current_date
                while new_date2 <= end_date_dt:
                    interval_type1 = interval_type
                    new_date1 = add_interval(new_date1, interval_type)
                    if new_date1 > end_date_dt:
                        interval_type1 = 'quarterly'
                        new_date1 = add_interval(new_date1, interval_type1)
                        if new_date1 > end_date_dt:
                            interval_type1 = 'monthly'
                            new_date1 = add_interval(new_date1, interval_type1)
                            if new_date1 > end_date_dt:
                                interval_type1 = 'weekly'
                                new_date1 = add_interval(new_date1, interval_type1)
                                if new_date1 > end_date_dt:
                                    interval_type1 = 'daily'
                                    new_date1 = add_interval(new_date1, interval_type1)
                    total_hours = calculate_total_hours(new_date1, interval_type1)
                    adjusted_hours = total_hours
                    if remove_weekends:
                        weekend_hours = calculate_weekend_hours(new_date1, interval_type1)
                        adjusted_hours -= weekend_hours
                    holiday_hours = calculate_holiday_hours(new_date1, interval_type1)
                    adjusted_hours -= holiday_hours
                    pto_hours = calculate_pto_hours(interval_type1)
                    adjusted_hours -= pto_hours
                    adjusted_hours = max(adjusted_hours, 0)
                    total_adjusted_hours += adjusted_hours
                    new_date2 = new_date1
            current_date = add_interval(current_date, interval_type)
        return {
            'initial_entities': initial_entities,
            'unit_arrival_pattern': unit_arrival_pattern,
            'total_adjusted_hours': total_adjusted_hours
        }

    def processor_dependency_graphs(self):
        """
        Visualize processor dependency graphs by p-level.
        """
        if not self.results:
            logger.warning("No results available. Run the simulation first.")
            return
        last_p_level_nodes = set()
        undirected_graph = nx.Graph()
        p_levels = set()
        for path_name, path_data in self.paths_info.items():
            processing_times = path_data['processing_times']
            G = path_data['graph']
            for node in G.nodes():
                if node in processing_times:
                    p_times = processing_times[node].get('processing_time', {})
                    if isinstance(p_times, dict):
                        p_levels.update(p_times.keys())
                    else:
                        p_levels.update([1, 2])
                    undirected_graph.add_node(node)
                    for succ in G.successors(node):
                        undirected_graph.add_edge(node, succ)
                    p_level_succs = processing_times[node].get('p_level_successors', {})
                    for succ_list in p_level_succs.values():
                        for succ in succ_list:
                            undirected_graph.add_edge(node, succ)
        print("Plotting the undirected graph with all nodes and edges across P-levels.")
        pos = nx.spring_layout(undirected_graph)
        plt.figure(figsize=(12, 10))
        nx.draw(undirected_graph, pos, with_labels=True, node_color='lightblue', edge_color='gray',
                node_size=1500, font_size=10)
        plt.title('Undirected Processor Dependency Graph (All P-levels)')
        plt.show()
        sorted_p_levels = sorted(p_levels)
        for idx, p_level in enumerate(sorted_p_levels):
            print(f"Plotting subgraph for p-level {p_level}")
            p_level_graph = nx.DiGraph()
            for path_name, path_data in self.paths_info.items():
                processing_times = path_data['processing_times']
                G = path_data['graph']
                for node in processing_times:
                    p_times = processing_times[node].get('processing_time', {})
                    if isinstance(p_times, dict):
                        if p_level in p_times:
                            p_level_graph.add_node(node)
                    else:
                        p_level_graph.add_node(node)
                for node in p_level_graph.nodes():
                    if node in processing_times:
                        p_level_succs = processing_times[node].get('p_level_successors', {})
                        successors = p_level_succs.get(p_level, None)
                        if successors is not None:
                            for succ in successors:
                                if succ in p_level_graph.nodes():
                                    p_level_graph.add_edge(node, succ)
                        else:
                            for succ in G.successors(node):
                                if succ in p_level_graph.nodes():
                                    p_level_graph.add_edge(node, succ)
            current_p_level_nodes = set(p_level_graph.nodes())
            new_nodes = current_p_level_nodes - last_p_level_nodes
            removed_nodes = last_p_level_nodes - current_p_level_nodes
            remaining_nodes = current_p_level_nodes & last_p_level_nodes
            last_p_level_nodes = current_p_level_nodes
            node_colors = []
            for node in p_level_graph.nodes():
                if node in self.p_level_changers:
                    color = 'orange'
                elif node in self.path_changers:
                    color = 'green'
                elif node in self.priority_changers:
                    color = 'red'
                elif node in new_nodes:
                    color = 'lightgreen'
                elif node in remaining_nodes:
                    color = 'lightblue'
                else:
                    color = 'gray'
                node_colors.append(color)
            plt.figure(figsize=(12, 10))
            pos = nx.spring_layout(p_level_graph)
            nx.draw(p_level_graph, pos, with_labels=True, node_color=node_colors,
                    edge_color='black', node_size=1500, font_size=10)
            from matplotlib.patches import Patch
            legend_elements = [
                Patch(facecolor='lightgreen', edgecolor='black', label='New Node'),
                Patch(facecolor='lightblue', edgecolor='black', label='Existing Node'),
                Patch(facecolor='orange', edgecolor='black', label='P-Level Changer'),
                Patch(facecolor='green', edgecolor='black', label='Path Changer'),
                Patch(facecolor='red', edgecolor='black', label='Priority Changer')
            ]
            plt.legend(handles=legend_elements, loc='best')
            plt.title(f'Processor Dependency Graph for P-Level {p_level}')
            plt.show()

    def balance_parameters(self, param_map, k_max, error_constraint='either', excluded_processors=None):
        """
        Balances parameters into sets of sizes from 2 to k_max.

        Parameters:
        - param_map: Dictionary mapping parameter names to values
        - k_max: Maximum size of the sets
        - error_constraint: 'positive', 'negative', or 'either'
        - excluded_processors: List of processor names to exclude

        Returns:
        - Dictionary where keys are sizes from 2 to k_max, and values are lists of sets
        """
        if excluded_processors is None:
            excluded_processors = []
        parameters = list(param_map.items())
        if excluded_processors:
            parameters = [(name, value) for name, value in parameters if name not in excluded_processors]
        all_params = set(name for name, value in parameters)
        total_params = len(all_params)
        results = {}
        def get_combinations(parameters, k, error_constraint='either'):
            combos = list(combinations(parameters, k))
            combo_sums = []
            for combo in combos:
                names = [name for name, value in combo]
                total = sum(value for name, value in combo)
                if error_constraint == 'positive' and total < 0:
                    continue
                if error_constraint == 'negative' and total > 0:
                    continue
                combo_sums.append({'names': names, 'total': total, 'abs_total': abs(total)})
            combo_sums.sort(key=lambda x: x['abs_total'])
            return combo_sums
        def select_combinations(combo_sums):
            selected_sets = []
            used_params = set()
            remaining_params = set(all_params)
            while remaining_params:
                for combo in combo_sums:
                    names = combo['names']
                    new_params = set(names) & remaining_params
                    combo['new_params'] = len(new_params)
                combo_sums.sort(key=lambda x: (-x['new_params'], x['abs_total']))
                for combo in combo_sums:
                    if combo['new_params'] > 0:
                        selected_sets.append(combo)
                        used_params.update(combo['names'])
                        remaining_params -= set(combo['names'])
                        break
                else:
                    if not combo_sums:
                        break
                    combo_sums.sort(key=lambda x: x['abs_total'])
                    selected_sets.append(combo_sums[0])
                    used_params.update(combo_sums[0]['names'])
                    remaining_params -= set(combo_sums[0]['names'])
                    break
            return selected_sets
        for k in range(2, k_max + 1):
            combo_sums = get_combinations(parameters, k, error_constraint=error_constraint)
            selected_sets = select_combinations(combo_sums)
            sum_positives = sum(combo['total'] for combo in selected_sets if combo['total'] > 0)
            sum_negatives = sum(combo['total'] for combo in selected_sets if combo['total'] < 0)
            results[k] = {
                'sets': selected_sets,
                'sum_positives': sum_positives,
                'sum_negatives': sum_negatives
            }
        return results

    def balance_parameters_dp(self, param_map, k_max, error_constraint='either', excluded_processors=None):
        """
        Balances parameters into sets of sizes from 2 to k_max using dynamic programming.

        Parameters:
        - param_map: Dictionary mapping parameter names to values
        - k_max: Maximum size of the sets
        - error_constraint: 'positive', 'negative', or 'either'
        - excluded_processors: List of processor names to exclude

        Returns:
        - Dictionary where keys are sizes from 2 to k_max, and values are lists of sets
        """
        if excluded_processors is None:
            excluded_processors = []
        if excluded_processors:
            param_map = {name: value for name, value in param_map.items() if name not in excluded_processors}
        parameters = list(param_map.items())
        total_params = len(parameters)
        results = {}
        for k in range(2, k_max + 1):
            all_combos = list(combinations(parameters, k))
            dp = {}
            for combo in all_combos:
                names = tuple(name for name, value in combo)
                total = sum(value for name, value in combo)
                if error_constraint == 'positive' and total < 0:
                    continue
                if error_constraint == 'negative' and total > 0:
                    continue
                abs_total = abs(total)
                dp[names] = abs_total
            sorted_combos = sorted(dp.items(), key=lambda x: x[1])
            selected_sets = []
            used_params = set()
            remaining_params = set(name for name, value in parameters)
            for names, abs_total in sorted_combos:
                names_set = set(names)
                if names_set & remaining_params:
                    selected_sets.append({
                        'names': names,
                        'total': sum(param_map[name] for name in names)
                    })
                    used_params.update(names)
                    remaining_params -= names_set
                    if not remaining_params:
                        break
            if remaining_params:
                for names, abs_total in sorted_combos:
                    names_set = set(names)
                    if names_set & remaining_params:
                        selected_sets.append({
                            'names': names,
                            'total': sum(param_map[name] for name in names)
                        })
                        used_params.update(names)
                        remaining_params -= names_set
                        if not remaining_params:
                            break
            sum_positives = sum(combo['total'] for combo in selected_sets if combo['total'] > 0)
            sum_negatives = sum(combo['total'] for combo in selected_sets if combo['total'] < 0)
            results[k] = {
                'sets': selected_sets,
                'sum_positives': sum_positives,
                'sum_negatives': sum_negatives
            }
        return results

def system_Optimization_model(
    params,
    mDaysShiftHours,
    wc24_3rdShiftAdj,
    n_entities,
    start_date,
    interval_type,
    end_date,
    initial_num_entities,
    arrival_pattern_num_entities,
    remove_weekends,
    holiday_dates,
    subtract_pto_hours_per_week,
    scale_days,
    remove_holiday_overlap,
    paths,
    p_level_changers,
    path_changers,
    priority_changers,
    exclude_processors,
    top_n,
    max_processors
):
    """
    Simulating processing and calculating idle times and dependencies using SimPy.

    Parameters:
    - params: List of adjustment parameters for processors
    - Various simulation configuration parameters

    Returns:
    - total_idle_time, total_processing_time, total_adjusted_hours
    """
    a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, a14, a15 = params
    sim = ProcessSimulation([], [], {}, {}, {}, {}, [], 0)
    arrival_results = sim.create_entity_arrival_schedule(
        start_date=start_date,
        interval_type=interval_type,
        end_date=end_date,
        initial_num_entities=initial_num_entities,
        arrival_pattern_num_entities=arrival_pattern_num_entities,
        remove_weekends=remove_weekends,
        holiday_dates=holiday_dates,
        subtract_pto_hours_per_week=subtract_pto_hours_per_week,
        scale_days=mDaysShiftHours,
        remove_holiday_overlap=remove_holiday_overlap
    )
    initial_entities = arrival_results['initial_entities']
    unit_arrival_pattern = arrival_results['unit_arrival_pattern']
    total_adjusted_hours = arrival_results['total_adjusted_hours']
    scalerWc05 = ((mDaysShiftHours + a1) / mDaysShiftHours)
    scalerWc03 = ((mDaysShiftHours + a2) / mDaysShiftHours)
    scalerWc21 = ((mDaysShiftHours + a3) / mDaysShiftHours)
    scalerWc02 = ((mDaysShiftHours + a4) / mDaysShiftHours)
    scalerWc08 = ((mDaysShiftHours + a5) / mDaysShiftHours)
    scalerWc07 = ((mDaysShiftHours + a6) / mDaysShiftHours)
    scalerWc06 = ((mDaysShiftHours + a7) / mDaysShiftHours)
    scalerWc22 = ((mDaysShiftHours + a8) / mDaysShiftHours)
    scalerWc23 = ((mDaysShiftHours + a9) / mDaysShiftHours)
    scalerWc11 = ((mDaysShiftHours + a10) / mDaysShiftHours)
    scalerWc19 = ((mDaysShiftHours + a11) / mDaysShiftHours)
    scalerWc15 = ((mDaysShiftHours + a12) / mDaysShiftHours)
    scalerWc17 = ((mDaysShiftHours + a13) / mDaysShiftHours)
    scalerWc24 = ((mDaysShiftHours + a14) / mDaysShiftHours)
    scalerWc20 = ((mDaysShiftHours + a15) / mDaysShiftHours)
    paths_info = {
        'Path1': {
            'processing_times': {
                'Start_Job': {'processing_time': 0, 'capacity': 1},
                'WC05': {'processing_time': 39/scalerWc05, 'capacity': 1},
                'WC03': {'processing_time': 39/scalerWc03, 'capacity': 1},
                'WC21': {'processing_time': 32/scalerWc21, 'capacity': 1},
                'WC02': {'processing_time': 32/scalerWc02, 'capacity': 1},
                'WC06': {'processing_time': 6/scalerWc06, 'capacity': 1},
                'WC24': {
                    'processing_time': {
                        1: 19/scalerWc24,
                        2: (((24-wc24_3rdShiftAdj)/scalerWc24) + (24-wc24_3rdShiftAdj) + (24-wc24_3rdShiftAdj))
                    },
                    'capacity': 2,
                    'p_level_successors': {
                        1: ['WC11'],
                        2: ['WC17']
                    }
                },
                'WC08': {'processing_time': 6/scalerWc08, 'capacity': 1},
                'WC07': {'processing_time': 32/scalerWc07, 'capacity': 1},
                'WC11': {'processing_time': 16/scalerWc11, 'capacity': 1},
                'WC19': {'processing_time': 32/scalerWc19, 'capacity': 1},
                'WC22': {'processing_time': 13/scalerWc22, 'capacity': 1},
                'WC23': {'processing_time': 32/scalerWc23, 'capacity': 1},
                'WC15': {'processing_time': 54/scalerWc15, 'capacity': 1},
                'WC17': {
                    'processing_time': {
                        1: 35/scalerWc17,
                        2: 13/scalerWc17
                    },
                    'capacity': 2,
                    'p_level_successors': {
                        1: ['WC20'],
                        2: ['Units_Delivered']
                    }
                },
                'WC20': {
                    'processing_time': 45/scalerWc20,
                    'capacity': 1,
                    'p_level_successors': {2: ['WC24']}
                },
                'Units_Delivered': {'processing_time': 0, 'capacity': 1}
            },
            'graph': nx.DiGraph([
                ('Start_Job', 'WC05'), ('Start_Job', 'WC02'), ('Start_Job', 'WC08'),
                ('Start_Job', 'WC22'), ('Start_Job', 'WC23'), ('WC05', 'WC03'),
                ('WC03', 'WC21'), ('WC21', 'WC06'), ('WC02', 'WC06'),
                ('WC06', 'WC24'), ('WC24', 'WC11'), ('WC24', 'WC17'),
                ('WC08', 'WC07'), ('WC07', 'WC11'), ('WC22', 'WC11'),
                ('WC11', 'WC19'), ('WC19', 'WC15'), ('WC23', 'WC15'),
                ('WC15', 'WC17'), ('WC17', 'WC20'), ('WC17', 'Units_Delivered')
            ])
        }
    }
    simulation = ProcessSimulation(
        initial_entities=initial_entities,
        unit_arrival_pattern=unit_arrival_pattern,
        paths_info=paths_info,
        p_level_changers=p_level_changers,
        path_changers=path_changers,
        priority_changers=priority_changers,
        exclude_processors=exclude_processors,
        n_entities=n_entities,
        top_n=top_n
    )
    results = simulation.run()
    total_idle_time = sum(results['idle_times'].values())
    total_processing_time = results['total_processing_time']
    return total_idle_time, total_processing_time, total_adjusted_hours

def optimize_parameters(
    initial_params,
    bounds,
    fixed_params,
    increments,
    integer_indices,
    total_adjusted_hours,
    mDaysShiftHours,
    wc24_3rdShiftAdj,
    n_entities,
    start_date,
    interval_type,
    end_date,
    initial_num_entities,
    arrival_pattern_num_entities,
    remove_weekends,
    holiday_dates,
    subtract_pto_hours_per_week,
    scale_days,
    remove_holiday_overlap,
    paths,
    p_level_changers,
    path_changers,
    priority_changers,
    exclude_processors,
    top_n,
    max_processors
):
    """
    Optimize parameters for the simulation model.

    Parameters:
    - initial_params: Starting parameter values
    - bounds: Min/max bounds for each parameter
    - fixed_params: Parameters that should not be optimized
    - And various other simulation configuration parameters

    Returns:
    - Optimized parameter values
    """
    from scipy.optimize import minimize
    def objective(params):
        params = np.array(params)
        for idx, val in fixed_params.items():
            params[idx] = val
        for idx in integer_indices:
            increment = increments.get(idx, 1)
            params[idx] = np.round(params[idx] / increment) * increment
        total_idle_time, _, _ = system_Optimization_model(
            params,
            mDaysShiftHours,
            wc24_3rdShiftAdj,
            n_entities,
            start_date,
            interval_type,
            end_date,
            initial_num_entities,
            arrival_pattern_num_entities,
            remove_weekends,
            holiday_dates,
            subtract_pto_hours_per_week,
            scale_days,
            remove_holiday_overlap,
            paths,
            p_level_changers,
            path_changers,
            priority_changers,
            exclude_processors,
            top_n,
            max_processors
        )
        return -total_idle_time
    def constraint_total_time(params):
        params = np.array(params)
        for idx, val in fixed_params.items():
            params[idx] = val
        for idx in integer_indices:
            increment = increments.get(idx, 1)
            params[idx] = np.round(params[idx] / increment) * increment
        _, total_processing_time, _ = system_Optimization_model(
            params,
            mDaysShiftHours,
            wc24_3rdShiftAdj,
            n_entities,
            start_date,
            interval_type,
            end_date,
            initial_num_entities,
            arrival_pattern_num_entities,
            remove_weekends,
            holiday_dates,
            subtract_pto_hours_per_week,
            scale_days,
            remove_holiday_overlap,
            paths,
            p_level_changers,
            path_changers,
            priority_changers,
            exclude_processors,
            top_n,
            max_processors
        )
        return total_adjusted_hours - total_processing_time
    constraints = [{'type': 'ineq', 'fun': constraint_total_time}]
    adjusted_bounds = []
    for i, (lower, upper) in enumerate(bounds):
        if i in fixed_params:
            adjusted_bounds.append((fixed_params[i], fixed_params[i]))
        else:
            adjusted_bounds.append((lower, upper))
    result = minimize(
        objective,
        initial_params,
        bounds=adjusted_bounds,
        constraints=constraints,
        method='SLSQP',
        options={'maxiter': 100}
    )
    optimized_params = result.x
    for idx, val in fixed_params.items():
        optimized_params[idx] = val
    for idx in integer_indices:
        increment = increments.get(idx, 1)
        optimized_params[idx] = np.round(optimized_params[idx] / increment) * increment
    return optimized_params

def main():
    """
    Main function to run the SimPy simulation.
    """
    start_time = time.time()
    n_entities = 189
    mDaysShiftHours = 6.5
    wc24_3rdShiftAdj = 11
    start_date = "2025-01-01"
    interval_type = "monthly"
    end_date = "2029-05-25"
    initial_num_entities = 1
    arrival_pattern_num_entities = 4
    remove_weekends = True
    holiday_dates = [
        "2025-01-01", "2025-05-25", "2025-07-04", "2025-09-01", "2025-11-23",
        "2025-11-24", "2025-12-24", "2025-12-25", "2025-12-26", "2025-12-27",
        "2025-12-28", "2025-12-29", "2026-01-01", "2026-05-25", "2026-07-04",
    ]
    subtract_pto_hours_per_week = 3.5 * 0.9
    remove_holiday_overlap = True
    exclude_processors = ['Start_Job', 'Units_Delivered']
    paths = ['Path1']
    sim = ProcessSimulation([], [], {}, n_entities=0)
    arrival_results = sim.create_entity_arrival_schedule(
        start_date=start_date,
        interval_type=interval_type,
        end_date=end_date,
        initial_num_entities=initial_num_entities,
        arrival_pattern_num_entities=arrival_pattern_num_entities,
        remove_weekends=remove_weekends,
        holiday_dates=holiday_dates,
        subtract_pto_hours_per_week=subtract_pto_hours_per_week,
        scale_days=mDaysShiftHours,
        remove_holiday_overlap=remove_holiday_overlap
    )
    initial_entities = arrival_results['initial_entities']
    unit_arrival_pattern = arrival_results['unit_arrival_pattern']
    total_adjusted_hours = arrival_results['total_adjusted_hours']
    print("\nTotal adjusted hours:", total_adjusted_hours)
    adjusted_hours_per_entity = total_adjusted_hours / n_entities
    print(f"Adjusted hours per entity: {adjusted_hours_per_entity:.2f}")
    a1 = 0
    a2 = 0
    a3 = 0
    a4 = 0
    a5 = 0
    a6 = 0
    a7 = 0
    a8 = 0
    a9 = 0
    a10 = 0
    a11 = 0
    a12 = 0
    a13 = 0
    a14 = 0
    a15 = 0
    scalerWc05 = (mDaysShiftHours + a1) / mDaysShiftHours
    scalerWc03 = (mDaysShiftHours + a2) / mDaysShiftHours
    scalerWc21 = (mDaysShiftHours + a3) / mDaysShiftHours
    scalerWc02 = (mDaysShiftHours + a4) / mDaysShiftHours
    scalerWc08 = (mDaysShiftHours + a5) / mDaysShiftHours
    scalerWc07 = (mDaysShiftHours + a6) / mDaysShiftHours
    scalerWc06 = (mDaysShiftHours + a7) / mDaysShiftHours
    scalerWc22 = (mDaysShiftHours + a8) / mDaysShiftHours
    scalerWc23 = (mDaysShiftHours + a9) / mDaysShiftHours
    scalerWc11 = (mDaysShiftHours + a10) / mDaysShiftHours
    scalerWc19 = (mDaysShiftHours + a11) / mDaysShiftHours
    scalerWc15 = (mDaysShiftHours + a12) / mDaysShiftHours
    scalerWc17 = (mDaysShiftHours + a13) / mDaysShiftHours
    scalerWc24 = (mDaysShiftHours + a14) / mDaysShiftHours
    scalerWc20 = (mDaysShiftHours + a15) / mDaysShiftHours
    p_level_changers = {
        'WC20': lambda p: 2 if p == 1 else 1,
        'WC24': lambda p: p,
        'WC17': lambda p: p
    }
    path_changers = {}
    priority_changers = {
        'WC20': lambda priority: 0,
        'WC24': lambda priority: priority,
        'WC17': lambda priority: priority
    }
    paths_info = {
        'Path1': {
            'processing_times': {
                'Start_Job': {'processing_time': 0, 'capacity': 1},
                'WC05': {'processing_time': 39/scalerWc05, 'capacity': 1},
                'WC03': {'processing_time': 39/scalerWc03, 'capacity': 1},
                'WC21': {'processing_time': 32/scalerWc21, 'capacity': 1},
                'WC02': {'processing_time': 32/scalerWc02, 'capacity': 1},
                'WC06': {'processing_time': 6/scalerWc06, 'capacity': 1},
                'WC24': {
                    'processing_time': {
                        1: 19/scalerWc24,
                        2: (((24-wc24_3rdShiftAdj)/scalerWc24) + (24-wc24_3rdShiftAdj) + (24-wc24_3rdShiftAdj))
                    },
                    'capacity': 2,
                    'p_level_successors': {
                        1: ['WC11'],
                        2: ['WC17']
                    }
                },
                'WC08': {'processing_time': 6/scalerWc08, 'capacity': 1},
                'WC07': {'processing_time': 32/scalerWc07, 'capacity': 1},
                'WC11': {'processing_time': 16/scalerWc11, 'capacity': 1},
                'WC19': {'processing_time': 32/scalerWc19, 'capacity': 1},
                'WC22': {'processing_time': 13/scalerWc22, 'capacity': 1},
                'WC23': {'processing_time': 32/scalerWc23, 'capacity': 1},
                'WC15': {'processing_time': 54/scalerWc15, 'capacity': 1},
                'WC17': {
                    'processing_time': {
                        1: 35/scalerWc17,
                        2: 13/scalerWc17
                    },
                    'capacity': 2,
                    'p_level_successors': {
                        1: ['WC20'],
                        2: ['Units_Delivered']
                    }
                },
                'WC20': {
                    'processing_time': 45/scalerWc20,
                    'capacity': 1,
                    'p_level_successors': {2: ['WC24']}
                },
                'Units_Delivered': {'processing_time': 0, 'capacity': 1}
            },
            'graph': nx.DiGraph([
                ('Start_Job', 'WC05'), ('Start_Job', 'WC02'), ('Start_Job', 'WC08'),
                ('Start_Job', 'WC22'), ('Start_Job', 'WC23'), ('WC05', 'WC03'),
                ('WC03', 'WC21'), ('WC21', 'WC06'), ('WC02', 'WC06'),
                ('WC06', 'WC24'), ('WC24', 'WC11'), ('WC24', 'WC17'),
                ('WC08', 'WC07'), ('WC07', 'WC11'), ('WC22', 'WC11'),
                ('WC11', 'WC19'), ('WC19', 'WC15'), ('WC23', 'WC15'),
                ('WC15', 'WC17'), ('WC17', 'WC20'), ('WC17', 'Units_Delivered')
            ])
        }
    }
    print("Running simulation with initial parameters...")
    simulation = ProcessSimulation(
        initial_entities=initial_entities,
        unit_arrival_pattern=unit_arrival_pattern,
        paths_info=paths_info,
        p_level_changers=p_level_changers,
        path_changers=path_changers,
        priority_changers=priority_changers,
        exclude_processors=exclude_processors,
        n_entities=n_entities
    )
    results = simulation.run()
    simulation.plot_processing_schedule()
    simulation.plot_results()
    simulation.plot_results_individual_processors()
    simulation.processor_dependency_graphs()
    initial_params = [2, 2, 0.5, 0, -5, 0.5, -5, -3.5, 0.5, -3.0, 0.5, 5, -1, 0, 3.5]
    bounds = [(-6.4, 6.5)] * 15
    fixed_params = {3: 0, 13: 0}
    increments = {}
    integer_indices = []
    print("\nOptimizing parameters...")
    optimized_params = optimize_parameters(
        initial_params,
        bounds,
        fixed_params,
        increments,
        integer_indices,
        total_adjusted_hours,
        mDaysShiftHours,
        wc24_3rdShiftAdj,
        n_entities,
        start_date,
        interval_type,
        end_date,
        initial_num_entities,
        arrival_pattern_num_entities,
        remove_weekends,
        holiday_dates,
        subtract_pto_hours_per_week,
        mDaysShiftHours,
        remove_holiday_overlap,
        paths,
        p_level_changers,
        path_changers,
        priority_changers,
        exclude_processors,
        0,
        1
    )
    print("\nOptimized Parameters:")
    optimized_param_map = {}
    for i, val in enumerate(optimized_params):
        param_name = f"a{i+1}"
        print(f"{param_name} = {val}")
        if i == 0:
            optimized_param_map['WC05'] = val
        elif i == 1:
            optimized_param_map['WC03'] = val
        elif i == 2:
            optimized_param_map['WC21'] = val
        elif i == 3:
            optimized_param_map['WC02'] = val
        elif i == 4:
            optimized_param_map['WC08'] = val
        elif i == 5:
            optimized_param_map['WC07'] = val
        elif i == 6:
            optimized_param_map['WC06'] = val
        elif i == 7:
            optimized_param_map['WC22'] = val
        elif i == 8:
            optimized_param_map['WC23'] = val
        elif i == 9:
            optimized_param_map['WC11'] = val
        elif i == 10:
            optimized_param_map['WC19'] = val
        elif i == 11:
            optimized_param_map['WC15'] = val
        elif i == 12:
            optimized_param_map['WC17'] = val
        elif i == 13:
            optimized_param_map['WC24'] = val
        elif i == 14:
            optimized_param_map['WC20'] = val
    print("\nRunning simulation with optimized parameters...")
    a1 = optimized_params[0]
    a2 = optimized_params[1]
    a3 = optimized_params[2]
    a4 = optimized_params[3]
    a5 = optimized_params[4]
    a6 = optimized_params[5]
    a7 = optimized_params[6]
    a8 = optimized_params[7]
    a9 = optimized_params[8]
    a10 = optimized_params[9]
    a11 = optimized_params[10]
    a12 = optimized_params[11]
    a13 = optimized_params[12]
    a14 = optimized_params[13]
    a15 = optimized_params[14]
    scalerWc05 = (mDaysShiftHours + a1) / mDaysShiftHours
    scalerWc03 = (mDaysShiftHours + a2) / mDaysShiftHours
    scalerWc21 = (mDaysShiftHours + a3) / mDaysShiftHours
    scalerWc02 = (mDaysShiftHours + a4) / mDaysShiftHours
    scalerWc08 = (mDaysShiftHours + a5) / mDaysShiftHours
    scalerWc07 = (mDaysShiftHours + a6) / mDaysShiftHours
    scalerWc06 = (mDaysShiftHours + a7) / mDaysShiftHours
    scalerWc22 = (mDaysShiftHours + a8) / mDaysShiftHours
    scalerWc23 = (mDaysShiftHours + a9) / mDaysShiftHours
    scalerWc11 = (mDaysShiftHours + a10) / mDaysShiftHours
    scalerWc19 = (mDaysShiftHours + a11) / mDaysShiftHours
    scalerWc15 = (mDaysShiftHours + a12) / mDaysShiftHours
    scalerWc17 = (mDaysShiftHours + a13) / mDaysShiftHours
    scalerWc24 = (mDaysShiftHours + a14) / mDaysShiftHours
    scalerWc20 = (mDaysShiftHours + a15) / mDaysShiftHours
    optimized_paths_info = {
        'Path1': {
            'processing_times': {
                'Start_Job': {'processing_time': 0, 'capacity': 1},
                'WC05': {'processing_time': 39/scalerWc05, 'capacity': 1},
                'WC03': {'processing_time': 39/scalerWc03, 'capacity': 1},
                'WC21': {'processing_time': 32/scalerWc21, 'capacity': 1},
                'WC02': {'processing_time': 32/scalerWc02, 'capacity': 1},
                'WC06': {'processing_time': 6/scalerWc06, 'capacity': 1},
                'WC24': {
                    'processing_time': {
                        1: 19/scalerWc24,
                        2: (((24-wc24_3rdShiftAdj)/scalerWc24) + (24-wc24_3rdShiftAdj) + (24-wc24_3rdShiftAdj))
                    },
                    'capacity': 2,
                    'p_level_successors': {
                        1: ['WC11'],
                        2: ['WC17']
                    }
                },
                'WC08': {'processing_time': 6/scalerWc08, 'capacity': 1},
                'WC07': {'processing_time': 32/scalerWc07, 'capacity': 1},
                'WC11': {'processing_time': 16/scalerWc11, 'capacity': 1},
                'WC19': {'processing_time': 32/scalerWc19, 'capacity': 1},
                'WC22': {'processing_time': 13/scalerWc22, 'capacity': 1},
                'WC23': {'processing_time': 32/scalerWc23, 'capacity': 1},
                'WC15': {'processing_time': 54/scalerWc15, 'capacity': 1},
                'WC17': {
                    'processing_time': {
                        1: 35/scalerWc17,
                        2: 13/scalerWc17
                    },
                    'capacity': 2,
                    'p_level_successors': {
                        1: ['WC20'],
                        2: ['Units_Delivered']
                    }
                },
                'WC20': {
                    'processing_time': 45/scalerWc20,
                    'capacity': 1,
                    'p_level_successors': {2: ['WC24']}
                },
                'Units_Delivered': {'processing_time': 0, 'capacity': 1}
            },
            'graph': nx.DiGraph([
                ('Start_Job', 'WC05'), ('Start_Job', 'WC02'), ('Start_Job', 'WC08'),
                ('Start_Job', 'WC22'), ('Start_Job', 'WC23'), ('WC05', 'WC03'),
                ('WC03', 'WC21'), ('WC21', 'WC06'), ('WC02', 'WC06'),
                ('WC06', 'WC24'), ('WC24', 'WC11'), ('WC24', 'WC17'),
                ('WC08', 'WC07'), ('WC07', 'WC11'), ('WC22', 'WC11'),
                ('WC11', 'WC19'), ('WC19', 'WC15'), ('WC23', 'WC15'),
                ('WC15', 'WC17'), ('WC17', 'WC20'), ('WC17', 'Units_Delivered')
            ])
        }
    }
    optimized_simulation = ProcessSimulation(
        initial_entities=initial_entities,
        unit_arrival_pattern=unit_arrival_pattern,
        paths_info=optimized_paths_info,
        p_level_changers=p_level_changers,
        path_changers=path_changers,
        priority_changers=priority_changers,
        exclude_processors=exclude_processors,
        n_entities=n_entities
    )
    optimized_results = optimized_simulation.run()
    optimized_simulation.plot_processing_schedule()
    optimized_simulation.plot_results()
    optimized_simulation.plot_results_individual_processors()
    print("\nBalancing parameters...")
    excluded_processors_for_balance = ['WC02', 'WC24']
    for error_constraint in ['either', 'negative']:
        print(f"\nParameter balancing with {error_constraint} constraint:")
        print("\nStandard parameter balancing:")
        results_bp = optimized_simulation.balance_parameters(
            optimized_param_map,
            k_max=15,
            error_constraint=error_constraint,
            excluded_processors=excluded_processors_for_balance
        )
        for k in results_bp:
            print(f"\nSets of size {k}:")
            total_sets = results_bp[k]['sets']
            for idx, combo in enumerate(total_sets):
                names = combo['names']
                total = combo['total']
                print(f"  Set {idx + 1}: {names}, Net Value: {total}")
            print(f"  Sum of Positives: {results_bp[k]['sum_positives']}")
            print(f"  Sum of Negatives: {results_bp[k]['sum_negatives']}")
        print("\nDP parameter balancing:")
        results_bp_dp = optimized_simulation.balance_parameters_dp(
            optimized_param_map,
            k_max=15,
            error_constraint=error_constraint,
            excluded_processors=excluded_processors_for_balance
        )
        for k in results_bp_dp:
            print(f"\nSets of size {k}:")
            total_sets = results_bp_dp[k]['sets']
            for idx, combo in enumerate(total_sets):
                names = combo['names']
                total = combo['total']
                print(f"  Set {idx + 1}: {names}, Net Value: {total}")
            print(f"  Sum of Positives: {results_bp_dp[k]['sum_positives']}")
            print(f"  Sum of Negatives: {results_bp_dp[k]['sum_negatives']}")
    print("\nSaving results to CSV files...")
    file_path01 = 'idle_combinations.csv'
    optimized_simulation.save_joint_times_to_csv(output_file=file_path01)
    file_path02 = 'clean_idle_combinations.csv'
    optimized_simulation.clean_idle_combinations_csv(
        input_file=file_path01,
        output_file=file_path02,
        excluded_processors=exclude_processors
    )
    all_processors = set()
    for path_data in optimized_paths_info.values():
        all_processors.update(path_data['processing_times'].keys())
    all_processors = sorted(all_processors - set(exclude_processors))
    k = 1
    optimal_pools = optimized_simulation.construct_optimal_pools(
        file_path02,
        k,
        1000000,
        excluded_processors=['Start_Job', 'Units_Delivered']
    )
    file_path03 = 'optimal_pools.csv'
    optimized_simulation.save_optimal_pools_to_csv(optimal_pools, output_file=file_path03)
    file_path04 = 'processor_schedules_dependencies.csv'
    optimized_simulation.save_processor_schedule_and_dependencies_to_csv(output_file=file_path04)
    end_time = time.time()
    runtime = end_time - start_time
    print(f"\nRuntime: {runtime:.2f} seconds")

if __name__ == "__main__":
    main()